In [1]:
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
from modelscope.outputs import OutputKeys
import cv2

img = 'https://modelscope.oss-cn-beijing.aliyuncs.com/test/images/noisy-demo-0.png'
image_denoise_pipeline = pipeline(Tasks.image_denoising, 'damo/cv_nafnet_image-denoise_sidd')
result = image_denoise_pipeline(img)[OutputKeys.OUTPUT_IMG]
cv2.imwrite('0421_result.png', result)

2023-04-21 11:29:00,422 - modelscope - INFO - PyTorch version 1.11.0+cu113 Found.
2023-04-21 11:29:00,427 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2023-04-21 11:29:00,457 - modelscope - INFO - Loading done! Current index file version is 1.4.3, with md5 25220b18bc5117531e8dfbca514f4e53 and a total number of 842 components indexed


2023-04-21 11:29:01.158488: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2023-04-21 11:29:02,478 - modelscope - INFO - Model revision not specified, use the latest revision: v1.0.0
2023-04-21 11:29:05,536 - modelscope - INFO - initiate model from /mnt/workspace/.cache/modelscope/damo/cv_nafnet_image-denoise_sidd
2023-04-21 11:29:05,537 - modelscope - INFO - initiate model from location /mnt/workspace/.cache/modelscope/damo/cv_nafnet_image-denoise_sidd.
2023-04-21 11:29:05,542 - modelscope - INFO - initialize model from /mnt/workspace/.cache/modelscope/damo/cv_nafnet_image-denoise_sidd
2023-04-21 11:29:05,823 - modelscope - INFO - Loading NAFNet model from /mnt/workspace/.cache/modelscope/damo/cv_nafnet_image-denoise_sidd/pytorch_model.pt, with param key: [params].
2023-04-21 11:29:05,943 - modelscope - INFO - load model done.
2023-04-21 11:29:05,983 - modelscope - INFO - load image denoise model done


True

In [ ]:
训练

In [2]:
import os
import tempfile

from modelscope.hub.snapshot_download import snapshot_download
from modelscope.msdatasets import MsDataset
from modelscope.msdatasets.task_datasets.sidd_image_denoising import \
    SiddImageDenoisingDataset
from modelscope.trainers import build_trainer
from modelscope.utils.config import Config
from modelscope.utils.constant import DownloadMode, ModelFile

tmp_dir = './20230421'
if not os.path.exists(tmp_dir):  # 创建工作目录
    os.makedirs(tmp_dir)
model_id = 'damo/cv_nafnet_image-denoise_sidd'  # model_id
cache_path = snapshot_download(model_id, cache_dir='original')  # 下载模型文件、配置文件
config = Config.from_file(
    os.path.join(cache_path, ModelFile.CONFIGURATION))  # 读取配置文件
print(config)

# 修改配置文件
def cfg_modify_fn(cfg):
    cfg.train.dataloader.batch_size_per_gpu = 2  # batch size
    cfg.train.dataloader.workers_per_gpu = 4
    
    cfg.train.max_epochs = 1  # 最大的epoch数
    cfg.train.optimizer.lr = 1e-5  # 学习率
    return cfg

# 加载数据集
dataset_train = MsDataset.load(
    'SIDD',
    namespace='huizheng',
    subset_name='default',
    split='validation',
    download_mode=DownloadMode.REUSE_DATASET_IF_EXISTS)._hf_ds  # 从modelscope hub 下载数据
dataset_val = MsDataset.load(
    'SIDD',
    namespace='huizheng',
    subset_name='default',
    split='test',
    download_mode=DownloadMode.REUSE_DATASET_IF_EXISTS)._hf_ds  # 从modelscope hub下载数据

dataset_train = SiddImageDenoisingDataset(
            dataset_train, config.dataset, is_train=True)  # 转为标准的pytorch map-style 数据集
dataset_val = SiddImageDenoisingDataset(
    dataset_val, config.dataset, is_train=False)

kwargs = dict(
    model=model_id,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    work_dir=tmp_dir, # 工作目录
    cfg_modify_fn=cfg_modify_fn)
trainer = build_trainer(default_args=kwargs)
trainer.train()

2023-04-21 11:33:02,159 - modelscope - WARNING - The reference has been Deprecated in modelscope v1.4.0+, please use `from modelscope.msdatasets.dataset_cls.custom_datasets import SiddImageDenoisingDataset`
2023-04-21 11:33:02,508 - modelscope - INFO - Model revision not specified, use the latest revision: v1.0.0
Downloading: 100%|██████████| 12.2M/12.2M [00:27<00:00, 457kB/s] 
Downloading: 100%|██████████| 12.2M/12.2M [00:02<00:00, 5.41MB/s]
Downloading: 100%|██████████| 26.2M/26.2M [01:06<00:00, 415kB/s] 
Downloading: 100%|██████████| 26.2M/26.2M [00:12<00:00, 2.20MB/s]
Downloading: 100%|██████████| 12.7M/12.7M [00:11<00:00, 1.21MB/s]
Downloading: 100%|██████████| 12.6M/12.6M [00:08<00:00, 1.47MB/s]
Downloading: 100%|██████████| 25.1M/25.1M [00:24<00:00, 1.07MB/s]
Downloading: 100%|██████████| 25.1M/25.1M [00:20<00:00, 1.26MB/s]
Downloading: 100%|██████████| 16.6M/16.6M [00:26<00:00, 645kB/s] 
Downloading: 100%|██████████| 16.5M/16.5M [00:18<00:00, 918kB/s] 
Downloading: 100%|███████

Config (path: original/damo/cv_nafnet_image-denoise_sidd/configuration.json): {'framework': 'pytorch', 'task': 'image-denoising', 'pipeline': {'type': 'nafnet-image-denoise'}, 'model': {'type': 'nafnet', 'network_g': {'img_channel': 3, 'width': 32, 'middle_blk_num': 12, 'enc_blk_nums': [2, 2, 4, 8], 'dec_blk_nums': [2, 2, 2, 2]}}, 'dataset': {'name': 'SIDD', 'type': 'PairedImageDataset', 'dataroot_gt': 'data/SIDD_example/', 'dataroot_lq': 'data/SIDD_example/', 'filename_tmpl': '{}', 'scale': 1, 'gt_size': 256, 'use_flip': False, 'use_rot': False}, 'preprocessor': {'type': 'image-denoise-preprocessor'}, 'train': {'dataloader': {'batch_size_per_gpu': 4, 'workers_per_gpu': 4, 'shuffle': True}, 'optimizer': {'type': 'AdamW', 'lr': 1e-05, 'weight_decay': 0, 'betas': [0.9, 0.9]}, 'lr_scheduler': {'type': 'CosineAnnealingLR', 'T_max': 200000, 'eta_min': 1e-07}, 'max_epochs': 2, 'hooks': [{'type': 'CheckpointHook', 'interval': 1}, {'type': 'TextLoggerHook', 'interval': 1}, {'type': 'IterTimerH

Using custom data configuration huizheng-c9e7ad15ae36a837


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

100% 

...

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset sidd downloaded and prepared to /root/.cache/modelscope/hub/datasets/huizheng/SIDD/master/meta/modelscope___dataset_builder/huizheng-e87194b0c2dd9e73/master/test. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

2023-04-21 11:39:46,720 - modelscope - INFO - Model revision not specified, use the latest revision: v1.0.0
2023-04-21 11:39:49,543 - modelscope - INFO - initialize model from /mnt/workspace/.cache/modelscope/damo/cv_nafnet_image-denoise_sidd
2023-04-21 11:39:49,826 - modelscope - INFO - Loading NAFNet model from /mnt/workspace/.cache/modelscope/damo/cv_nafnet_image-denoise_sidd/pytorch_model.pt, with param key: [params].
2023-04-21 11:39:49,986 - modelscope - INFO - load model done.
2023-04-21 11:39:50,031 - modelscope - INFO - ==========================Training Config Start==========================
2023-04-21 11:39:50,032 - modelscope - INFO - {
    "framework": "pytorch",
    "task": "image-denoising",
    "pipeline": {
        "type": "nafnet-image-denoise"
    },
    "model": {
        "type": "nafnet",
        "network_g": {
            "img_channel": 3,
            "width": 32,
            "middle_blk_num": 12,
            "enc_blk_nums": [
                2,
                2,

In [ ]:
评估

In [3]:
import os
import tempfile
import torch

from modelscope.hub.snapshot_download import snapshot_download
from modelscope.utils.config import Config
from modelscope.utils.constant import DownloadMode, ModelFile
from modelscope.trainers import build_trainer
from modelscope.msdatasets import MsDataset
from modelscope.msdatasets.task_datasets.sidd_image_denoising import \
    SiddImageDenoisingDataset

# 工作目录中包含model card中的所有文件（包括配置文件、模型文件）
tmp_dir = './20230421/output/'  # 如果训练过程中的工作目录为`./image_denoise/`，那么验证时的工作目录需要改为 './image_denoise/output/'
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)
model_id = 'damo/cv_nafnet_image-denoise_sidd'
# cache_path = snapshot_download(model_id)
# config = Config.from_file(
#     os.path.join(cache_path, ModelFile.CONFIGURATION))

# 因为cv_nafnet_image-denoise_sidd模型使用modelscope的trainer类进行训练，其自动保存的模型中每一层的关键字都会有一个`model.`的前缀，
# 我们需要在这里去掉这个前缀。
# 修改 tmp_dir/epoch_1.pth 或 tmp_dir/output/pytorch_model.pt 为模型能够直接加载的文件
def modify_checkpoint(ckpt_path, saved_path):
    input_ckpt = torch.load(ckpt_path)
    
    pretrained_dict = {k.replace('model.', ''): v for k, v in input_ckpt.items() if k.startswith('model.')}
    torch.save(pretrained_dict, saved_path)
    print('successly convert checkpoint!')

modify_checkpoint(tmp_dir+'pytorch_model.pt', tmp_dir+'pytorch_model.pt')

dataset_val = MsDataset.load(
    'SIDD',
    namespace='huizheng',
    subset_name='default',
    split='test',
    download_mode=DownloadMode.REUSE_DATASET_IF_EXISTS)._hf_ds
eval_dataset = SiddImageDenoisingDataset(
            dataset_val, config.dataset, is_train=False)
kwargs = dict(
    model=tmp_dir,  # 可指定为模型所谓目录，例如 tmp_dir
    train_dataset=None,
    eval_dataset=eval_dataset,
    work_dir=tmp_dir)
trainer = build_trainer(default_args=kwargs)
metric_values = trainer.evaluate()
print(metric_values)

successly convert checkpoint!


Using custom data configuration huizheng-e87194b0c2dd9e73
Found cached dataset sidd (/root/.cache/modelscope/hub/datasets/huizheng/SIDD/master/meta/modelscope___dataset_builder/huizheng-e87194b0c2dd9e73/master/test)


  0%|          | 0/1 [00:00<?, ?it/s]

2023-04-21 11:44:33,175 - modelscope - ERROR - Authentication token does not exist, failed to access model unknown which may not exist or may be                 private. Please login first.
2023-04-21 11:44:33,175 - modelscope - ERROR - Response details: b'{"Code":10010205001,"Message":"\xe8\x8e\xb7\xe5\x8f\x96\xe6\xa8\xa1\xe5\x9e\x8b\xe4\xbf\xa1\xe6\x81\xaf\xe5\xa4\xb1\xe8\xb4\xa5\xef\xbc\x8c\xe4\xbf\xa1\xe6\x81\xaf\xef\xbc\x9arecord not found","RequestId":"d4161bd3-dff6-11ed-abf4-0eecc5c9cb8a","Success":false}'
2023-04-21 11:44:33,244 - modelscope - ERROR - Authentication token does not exist, failed to access model unknown which may not exist or may be                 private. Please login first.
2023-04-21 11:44:33,244 - modelscope - ERROR - Response details: b'404 page not found'
2023-04-21 11:44:33,251 - modelscope - INFO - initialize model from ./20230421/output/
2023-04-21 11:44:33,971 - modelscope - INFO - Loading NAFNet model from ./20230421/output/pytorch_model.pt, with para

{'psnr': 39.151959537434855, 'ssim': 0.9559883548878133}


In [ ]:
测试图片

In [1]:
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
from modelscope.outputs import OutputKeys
from modelscope.pipelines.cv import ImageDenoisePipeline
import cv2

img = 'https://modelscope.oss-cn-beijing.aliyuncs.com/test/images/noisy-demo-0.png'
# image_denoise_pipeline = pipeline(Tasks.image_denoising, 'damo/cv_nafnet_image-denoise_sidd')
# image_denoise_pipeline = pipeline(Tasks.image_denoising, './20230421/output/')
image_denoise_pipeline = ImageDenoisePipeline('./20230421/output/')
result = image_denoise_pipeline(img)[OutputKeys.OUTPUT_IMG]
cv2.imwrite('0421_result2.png', result)

2023-04-22 19:08:39,389 - modelscope - INFO - PyTorch version 1.11.0+cu113 Found.
2023-04-22 19:08:39,395 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2023-04-22 19:08:39,471 - modelscope - INFO - No valid ast index found from /mnt/workspace/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2023-04-22 19:08:39,525 - modelscope - INFO - Loading done! Current index file version is 1.5.0, with md5 ca25ccc146d421f40d58ac06319b4460 and a total number of 860 components indexed
2023-04-22 19:08:40,814 - modelscope - INFO - initiate model from ./20230421/output/
2023-04-22 19:08:40,819 - modelscope - INFO - initiate model from location ./20230421/output/.
2023-04-22 19:08:40,830 - modelscope - INFO - initialize model from ./20230421/output/
2023-04-22 19:08:41,605 - modelscope - INFO - Loading NAFNet model from ./20230421/output/pytorch_model.pt, with param key: [params].
2023-04-22 19:08:41,725 - modelscope - INFO - load model done

True